# Initialize the SparkSession

Run the cells below to get a SparkSession.

In [ ]:
import findspark
findspark.init()
import pyspark

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [ ]:
from delta.tables import *
import pyspark.sql.functions as f

# Store data in delta format

The data we use is the COVID-19 daily report data from Johns Hopkins university. It's already download to `/home/jovyan/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/` as CSV files, with the filename formatted as the date in American format (mm-dd-yyyy). For now we will only load the April 2021 data.

After loading the CSV data, you need to write it in the delta format. Use `/tmp/deltalake/` to avoid permission issues.

- [PySpark cheat sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf)
- [Spark SQL documentation](https://spark.apache.org/docs/latest/api/python/reference/index.html)

In [ ]:
df = spark.read.format("csv").options(header='True', inferSchema='True', delimiter=',').load("/home/jovyan/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/04-*-2021.csv")
df.show()

In [ ]:
df.write.format("delta").save("/tmp/deltalake/")

Delta lake writes parquet files to location on disk. Can you find those files?

# Read data in delta format

After storing the data in delta format you can now read it. Try showing the first few rows.

In [ ]:
df = spark.read.format("delta").load("/tmp/deltalake/")
df.show(5)

# Clean up the data

We use the `FIPS`, `Admin2`, `Province_State`, `Country_Region` and `Last_Update` as the unique key of this dataset. For ease of use later on we want to clean up the nulls in `FIPS`, `Admin2` and `Province_State` column.

Do this using DeltaTable class with the update method. Replace nulls for strings with an empty string (`''`) and for numbers with `0`.

Hint: you can check for nulls with the `isNull()` method of `f.col()`.

- [Documentation of DeltaTable](https://docs.delta.io/latest/api/python/index.html).
- [Documentation of Spark SQL Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions).

In [ ]:
df.dtypes

In [ ]:
deltaTable = DeltaTable.forPath(spark, '/tmp/deltalake')

deltaTable.update( .... (complete this code)

In [ ]:
df.show(5)

Delta Lake keeps a history of you data. You can see the metadata of this history with the `history()` method of DeltaTable. Try it out below to see the effects of your updates.

You can revisit this after any changes to the delta table.

# Upsert new data

The Confirmed cases for Flevoland in the Netherlands turned out to be different. We've received a new file with the fix, it's stored in `/home/jovyan/delta-lake-workshop/files/04-07-2021.csv`.

- Clean the new data with the same rules as you cleaned the existing data in the deltalake (replace null with `''` or `0`).
- Upsert the new data with a merge into the deltalake. You can find an example [here](https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-merge). Remember the columns that make up the key.

# Time travel
In a data warehouse (or lakehouse) you often want to be able to time travel, so you can see what the world looked like at a certain moment in time. This feature is very useful for data rollbacks, auditing, and reproducing reports and analytics at any point in time.

Since history is kept, this feature is supported out of the box. You can find examples on how to access the history [here](https://docs.delta.io/latest/delta-batch.html#query-an-older-snapshot-of-a-table-time-travel).

# Bonus: What other useful features can you come up with?
Think about what else you can do with DeltaLake, experiment and show you code! Look at the inspiration below:

# Advanced Delta Lake: schema updates

We didn't think you would get here already, but if you're up for it you can take it one step further.

Remember that we only put April 2021 in our delta lake? The reason for this was that the data set has had quite a bit of schema drift in the last 1+ year. Delta Lake can deal with this, either manually (meh) or automatically (yay). Check out the examples [here](https://docs.delta.io/latest/delta-batch.html#update-table-schema).

We haven't tried this feature yet, so we're curious what your solution is.